In [1]:
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
from scipy.stats import mode
import operator
import functools 

In [2]:
df_2012 = pd.read_csv('BPI_Challenge_2012-training.csv')

In [3]:
df_2012

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


In [4]:
temp = []
for i,j in df_2012.iterrows():
    date_time_obj = datetime.datetime.strptime(j['event time:timestamp'], '%d-%m-%Y %H:%M:%S.%f')
    temp.append(date_time_obj)
df_2012["event time:time"] = temp 
df_2012

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,event time:time
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546,2011-10-01 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880,2011-10-01 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906,2011-10-01 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875,2011-10-01 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256,2011-10-01 08:08:58.256
...,...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361,2012-03-14 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361,2012-03-14 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187,2012-03-14 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299,2012-03-14 15:36:15.299


In [5]:


# Assign position number to each event
df_sort = df_2012.set_index(df_2012.groupby('case concept:name').cumcount(), append = True)
df_sort = df_sort.reset_index()
df_sort2 = df_sort[["level_1", "event concept:name"]]

# Find the most frequent event in each position using mode
df_new = df_sort2.groupby('level_1')['event concept:name'].apply(lambda x: mode(x)[0][0]).reset_index()
df_new = df_new.rename(columns={'level_1':'position', 'event concept:name':'Predicted event'})
df_new


df_sort = df_sort.set_index(df_sort.groupby('case concept:name').cumcount(), append = True)
df_sorted = df_sort.sort_values(['case concept:name','level_1'])#reset_index()
df_sorted

,,level_0,level_1,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,event time:time
0,0,0,0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546,2011-10-01 00:38:44.546
1,1,1,1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880,2011-10-01 00:38:44.880
2,2,2,2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906,2011-10-01 00:39:37.906
3,3,3,3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875,2011-10-01 00:39:38.875
89,4,89,4,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,01-10-2011 11:36:46.437,2011-10-01 11:36:46.437
...,...,...,...,...,...,...,...,...,...,...,...
199718,4,199718,4,44955422687236,206318,2012-02-03T17:07:38.334+01:00,5000,A_DECLINED,COMPLETE,03-02-2012 17:55:57.294,2012-02-03 17:55:57.294
199719,5,199719,5,44955422687237,206318,2012-02-03T17:07:38.334+01:00,5000,W_Afhandelen leads,COMPLETE,03-02-2012 17:56:00.998,2012-02-03 17:56:00.998
199654,0,199654,0,44959717654528,206321,2012-02-03T17:08:39.199+01:00,2000,A_SUBMITTED,COMPLETE,03-02-2012 17:08:39.200,2012-02-03 17:08:39.200
199655,1,199655,1,44959717654529,206321,2012-02-03T17:08:39.199+01:00,2000,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:08:39.459,2012-02-03 17:08:39.459


In [6]:
correct = {}
temp = []
previous = []
for count,row in df_sorted.iterrows():
    
    i = row["level_1"];   
    if i == 0:
            temp = row["case REG_DATE"]
            if len(temp) == 25:
                part1,part2 = temp.split("+")
                part1 = part1 + ".000+"
                temp = part1+part2
                print(temp)
            temp = temp.replace("T"," ")
            temp,_ = temp.split("+")
            date_time_obj = datetime.datetime.strptime(temp, '%Y-%m-%d %H:%M:%S.%f')
            calc  = row["event time:time"] - date_time_obj
    if i != 0:
            calc  = row["event time:time"] - previous["event time:time"]
    if i in correct:
                correct[i].append(calc)
    else:
                correct[i] = [calc]
    previous = row

correct

2011-10-02T16:53:33.000+02:00
2011-10-12T19:06:15.000+02:00
2011-10-31T22:40:39.000+01:00
2011-11-10T14:02:25.000+01:00
2011-11-13T11:42:58.000+01:00
2011-11-17T19:25:59.000+01:00
2011-11-20T10:09:26.000+01:00
2011-12-15T15:14:00.000+01:00
2011-12-15T17:07:49.000+01:00
2011-12-15T22:07:09.000+01:00
2012-01-08T21:24:48.000+01:00
2012-01-09T19:34:13.000+01:00
2012-01-12T21:44:51.000+01:00
2012-01-28T11:58:56.000+01:00


{0: [Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00.001000'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00.001000'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00.001000'),
  Timedelta('0 days 00:00:00.001000'),
  Timedelta('0 days 00:00:00.001000'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:00.001000'),
  Timedelta('0 days 00:00:00'),
  Timedelta('0 days 00:00:0

In [7]:
actual = []
for key in correct:
        actual.append(str(functools.reduce(operator.add, correct[key])/len(correct[key])))
actual
df_new["Expected time"] = actual

In [8]:
df_new

,position,Predicted event,Expected time
0,0,A_SUBMITTED,0 days 00:00:00.000319
1,1,A_PARTLYSUBMITTED,0 days 00:00:00.582640
2,2,W_Afhandelen leads,0 days 00:00:35.102779
3,3,W_Afhandelen leads,0 days 02:51:15.661069
4,4,W_Completeren aanvraag,0 days 03:15:52.910088
...,...,...,...
170,170,W_Nabellen incomplete dossiers,0 days 00:00:14.929000
171,171,W_Nabellen incomplete dossiers,0 days 00:26:08.870000
172,172,O_CANCELLED,0 days 00:05:04.951000
173,173,A_CANCELLED,0 days 00:00:00


In [9]:
#THIS IS WORK ON THE ERROR MEASUREMENT


correct = {}
temp = []
previous = []
for count,row in df_sorted.iterrows():
    if 
    i = row["level_1"]
    if df_new["Predicted event"][i] == row["event concept:name"]:
        if i != 0:
            calc  = row["event time:time"] - previous["event time:time"]
            if i in correct:
                correct[i].append(calc)
            else:
                correct[i] = [calc]
    previous = row

correct

SyntaxError: invalid syntax (<ipython-input-9-5648cccc9588>, line 8)

In [ ]:
import operator
import functools 

In [ ]:
#THIS IS WORK ON THE ERROR MEASUREMENT

In [ ]:
actual = []
for key in correct:
        print(functools.reduce(operator.add, correct[key]),len(correct[key]),max(correct[key]))

In [ ]:
actual